In [9]:
import pandas as pd
# import numpy as np
import re

# from tabula import read_pdf
# from tabulate import tabulate
# import pdfplumber
# import openpyxl

from Script import ExtractPDFTables, MapLinks2Excel

# Table Extraction and Mapping (SGD-GRI Link)

In [4]:
pdf_path = 'ESG-Frameworks/Mapping-Standards/SDG-GRI/sdg-gri.pdf'
page_range = [list(range(3, 73)), list(range(74, 99))]
area = [80.51, 90.42, 561.96, 814.18]

tableTCFD_GRI = ExtractPDFTables(pdf_path, page_range, area)
df = tableTCFD_GRI.getTablesSDG_GRI_2('ESG-Frameworks/ESG Data/SDG_info.csv')

Are you going to map GRI on SDG sheet? (yes or no):  no


## Mapping Collected Dataframe (SDG-GRI)

In [3]:
excelFilePath = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = "SDG's"

mapp = MapLinks2Excel(df, sheet, excelFilePath)
mapp.MapSDG_GRI()

"SDG's sheet from Excel file have bee mapped"

## Mapping Collected Dataframe (GRI-SDG)

In [5]:
excelFilePath = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = "GRI 2016"

mapp = MapLinks2Excel(df, sheet, excelFilePath)
mapp.MapGRI_SDG()

'GRI 2016 sheet from Excel file have bee mapped'

In [ ]:
sdg_df = pd.read_csv('ESG-Frameworks/ESG Data/SDG_info.csv')

In [ ]:
df

In [ ]:
df3 = pd.merge(df, sdg_df, on=['SDG Target'], how='right')

In [ ]:
df3[['SDG Target','GRI Available Business Disclosures', 'GRI Disclosure', 'SDG Description']]

In [ ]:
df

# Table Extraction and Mapping (COH4B-GRI Link)

In [10]:
pdf_path = 'ESG-Frameworks/Mapping-Standards/GRI-COH4B/gri-coh4b.pdf'
page_range = list(range(13, 22))


tableGRI_COH4B = ExtractPDFTables(pdf_path, page_range)
df = tableGRI_COH4B.getTablesGRI_COH4B()
df = tableGRI_COH4B.setHeaders(df, 0)
df = tableGRI_COH4B.headerSwap(df, 'A. COHBP & \ndefinition', None, 'id')
df = tableGRI_COH4B.addDot(df, 'id')



df = tableGRI_COH4B.extractDisclosures(df, 'D. GRI disclosures', 'GRI Standards', '[0-9]{3}-[0-9]{2}|[0-9]{3}-[0-9]{1}', re.search)



## Mapping Collected Dataframe (COH4B-GRI)

In [12]:
excelFilePath = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = "COH4B"

mapp = MapLinks2Excel(df, sheet, excelFilePath)
mapp.MapCOH4B_GRI()

'COH4B sheet from Excel file have bee mapped'

## Mapping Collected Dataframe (GRI-COH4B)

In [13]:
excelFilePath = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = "GRI 2016"

mapp = MapLinks2Excel(df, sheet, excelFilePath)
mapp.MapGRI_COH4B()

'GRI 2016 sheet from Excel file have bee mapped'

# Table Extraction and Mapping (TCFD-GRI Link)

In [ ]:
pdf_path = 'ESG-Frameworks/Mapping-Standards/TCFD-GRI/TCFD to GRI-2.pdf'
page_range = list(range(49, 59))


tableTCFD_GRI = ExtractPDFTables(pdf_path, page_range)
df = tableTCFD_GRI.getTablesTCFD_GRI()
df = tableTCFD_GRI.extractDisclosures2(df, 'Related \ncode/\nparagraph', 'GRI Standards', r'[0-9]{3}-[0-9]{2}|[0-9]{3}-[0-9]{1}', re.findall)
df = tableTCFD_GRI.extractDisclosures2(df, 'Recommended \nDisclosures \n(TCFD Framework)', 'id', r'\w+\s[abc]\.', re.findall)




# [a-zA-Z]\.

## Mapping Collected Dataframe (TCDF-GRI)

In [ ]:
excelFilePath = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = "TCFD"


mapp = MapLinks2Excel(df, sheet, excelFilePath)
mapp.mapTCFD_GRI()


## Mapping Collected Dataframe (GRI-TCDF)

In [ ]:
df = tableTCFD_GRI.df_gri_tcdf(df)

excelFilePath = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = "GRI 2016"


mapp = MapLinks2Excel(df, sheet, excelFilePath)
mapp.mapGRI_TCFD()

In [ ]:
df